### Exercise 7.10 (programming) 

**Q**

Devise a small off-policy prediction problem and use it to show that the off-policy learning algorithm using (7.13) and (7.2) is more data efficient than the simpler algorithm using (7.1) and (7.9).

**A**

Equation 7.13 (to calculate $G_{t:h}$ using the off-policy algorithm with the state values) is:

$$
G_{t:h} \doteq \rho_t (R_{t+1} + \gamma G_{t+1:h}) + (1 - \rho_t) V_{h-1}(S_t), \quad t < h < T \tag{a2}
$$

with $G_{h:h} = V_{h-1}(S_h)$.

Equation 7.2 is:

$$
V_{t+n} (S_t) \doteq V_{t+n-1} (S_t) + \alpha [G_{t:t+n} - V_{t+n-1} (S_t)], \quad 0 \leq t \lt T
$$

Equation 7.1 is:

$$
G_{t:t+n} \doteq R_{t+1} + \gamma R_{t+2} + ... + \gamma^{n-1} R_{t+n} + \gamma^n V_{t+n-1} (S_{t+n}), \quad 0 \leq t \lt T + n
$$

with $G_{t:t+n} \doteq G_t$ if $t + n \geq T$.

Equation 7.9 is:

$$
V_{t+n} (S_t) \doteq V_{t+n-1} (S_t) + \alpha \rho_{t:t+n-1} [G_{t:t+n} - V_{t+n-1} (S_t)], \quad 0 \leq t \lt T
$$

To demonstrate the algorithms, it will be used the same random walk experiment from Exercise 7.2, with the environment without actions, because it's just a prediction problem in which the objective is to converge $V$ into $v_*$.